# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
from IPython.display import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import tensorflow as tf
import tensorflow_datasets as tfds
warnings.filterwarnings("ignore")

In [10]:
os.chdir('/content/drive/My Drive/Colab Notebooks/머신러닝 교과서/data')

## 16.3.2 두 번째 프로젝트 : 텐서플로로 글자 단위의 언어 모델 구현

언어 모델링은 영어 문장 생성처럼 기계가 사람의 언어와 관련된 작업을 수행하도록 하는 흥미로운 애플리케이션이다.

이 분야에서 관심을 끄는 결과물 중 하나는 서스키버, 마틴, 힌튼의 작업이다.

앞으로 만들 모델의 입력은 텍스트 문서이다.

입력 문서와 비슷한 스타일로 새로운 텍스트를 생성하는 모델을 만드는 것이 목표이다.

글자 단위 언어 모델링에서 입력은 글자의 시퀀스로 나누어 한 번에 글자 하나씩 네트워크에 주입된다.

이 네트워크는 지금까지 본 글자와 함께 새로운 글자를 처리하여 다음 글자를 예측한다.

In [3]:
Image(url='https://git.io/JLdVE', width=700)

In [6]:
! curl -O http://www.gutenberg.org/files/1268/1268-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [11]:
#텍스트 읽고 전처리
with open('1268-0.txt', 'r', encoding='UTF8') as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('전체 길이:', len(text))
print('고유한 문자:', len(char_set))

566 1112916
전체 길이: 1112350
고유한 문자: 80


텍스트를 내려받고 전처리하여 총 111만 2,350개의 문자와 80개의 고유한 문자로 구성된 시퀀스를 얻었다.

하지만 대부분 신경망 라이브러리와 RNN 구현은 문자열 형태의 입력 데이터를 다룰 수 없다.

이 때문에 텍스트 데이터를 숫자 형태로 바꾸어야 한다.

또한, 모델의 출력 결과를 텍스트로 변환하는 역 매핑도 필요하다.

정수와 문자를 키와 값으로 연결한 딕셔너리로 역 매핑을 수행할 수도 있지만 인덱스와 고유 문자를 매핑한 넘파이 배열을 사용하는 것이 훨씬 효율적이다.

In [13]:
Image(url='https://git.io/JLdVz', width=700)

In [16]:
# 문자를 정수로 매핑하는 딕셔너리를 만드는 것과 넘파이 배열의 인덱싱을 사용하여 반대로 매핑하는 예
chars_sorted=sorted(char_set)
char2int={ch:i for i,ch in enumerate(chars_sorted)}
char_array=np.array(chars_sorted)

text_encoded=np.array(
    [char2int[ch] for ch in text],dtype=np.int32
)

print('인코딩된 텍스트 크기:',text_encoded.shape)

인코딩된 텍스트 크기: (1112350,)


In [18]:
print(text[:15], '     == 인코딩 ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == 디코딩 ==> ', ''.join(char_array[text_encoded[15:21]]))

THE MYSTERIOUS       == 인코딩 ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == 디코딩 ==>  ISLAND


넘파이 배열 text_encoded는 텍스트에 있는 모든 문자에 대한 인코딩 값을 담고 있다.

이 배열을 이용해 텐서플로 데이터셋을 만들겠다.

In [20]:
import tensorflow as tf


ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [19]:
Image(url='https://git.io/JLdVV', width=700)

텍스트 생성 모델을 구현하기 위해 먼저 시퀀스 길이를 40으로 자르겠다.

시퀀스 길이는 생성된 텍스트의 품질에 영향을 미친다.

긴 시퀀스가 더 의미 있는 문장을 만들 수 있다.

하지만, 짧은 시퀀스일 경우 모델이 대부분 문맥을 무시하고 개별 단어를 정확히 감지하는 게 초점을 맞출 수 있다.

긴 시퀀스가 보통 더 의미 있는 문장을 만들지만 긴 시퀀스에서 RNN 모델이 장기간 의존성을 감지하기 어렵다.

실제로 적절한 시퀀스 길이를 찾는 것은 경험적으로 평가해야 하는 하이퍼파라미터 최적화 문제이다.

배치 크기를 41로 하여 처음 40개의 원소는 입력 마지막 40개는 타겟으로 하겠다.

In [21]:
seq_length=40
chunk_size=seq_length+1
ds_chunks=ds_text_encoded.batch(chunk_size,drop_remainder=True)

def split_input_target(chunk):
  input_seq=chunk[:-1]
  target_seq=chunk[1:]
  return input_seq,target_seq

ds_sequences=ds_chunks.map(split_input_target)

In [22]:
#몇 개의 샘플을 확인해보겠다.
for example in ds_sequences.take(2):
    print('입력 (x):', repr(''.join(char_array[example[0].numpy()])))
    print('타깃 (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

입력 (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
타깃 (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

입력 (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
타깃 (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



In [23]:
#미니배치 만들기
BATCH_SIZE=64
BUFFER_SIZE=10000
ds=ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

문자 수준의 모델 만들기

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [25]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         20480     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


LSTM 층의 출력크기는 (None,None,512)로 랭크 3이다.

첫 번째 차원은 배치 차원이다.

두 번째 차원은 출력 시퀀스 길이고 마지막 차원은 은닉 유닛의 개수이다.

랭크 3의 출력을 만드는 이유는 LSTM 층을 만들 때 return_sequences=True로 지정했기 때문에

마지막 완전 연결 층을 activation=None으로 설정했다.

새로운 텍스트를 생성하기 위해 모델 예측 값에서 샘플링할 수 있도록 로짓 출력이 필요하기 때문에

In [27]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=1)

424/424 [==============================] - 377s 883ms/step - loss: 2.1315


평가 단계

이전 절에서 훈련한 RNN 모델은 각 문자에 대해 80개 크기의 로짓을 반환한다.

소프트맥스 함수를 사용해서 이 로짓을 쉽게 확률로 바꿀 수 있다.

이 확률을 사용하여 어떤 문자가 다음에 올지 결정한다.

간단히 가장 큰 로짓 값을 가진 우너소를 선택할 수 있다.

하지만 이 대신 출력에서 샘플링하려고 한다.

이렇게 하지 않으면 항상 동일한 텍스트를 만든다.

텐서플로에서 제공하는 tf.random.categorical 함수를 사용하여 범주형 분포에서 랜덤하게 샘플링할 수 있다.

입력 로짓이 [1,1,1]일 때 세 개의 범주 [0,1,2]에서 랜덤하게 샘플링 해보겠다.

In [30]:
tf.random.set_seed(1)
logits=[[1.0,1.0,1.0,]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples=tf.random.categorical(logits=logits,num_samples=10)

tf.print(samples.numpy())

확률: [0.33333334 0.33333334 0.33333334]
array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


In [31]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('확률:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

확률: [0.10650698 0.10650698 0.78698605]
array([[2, 0, 2, 2, 2, 0, 1, 2, 2, 0]])


로짓을 기반으로 문자를 생성할 수 있다.



In [32]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island hed seach a shatee of the prifters a miect carain of
thes issoppeeve, oncicated by lack gulon; aid recanient, hereing main the onchowed ancacimed. It with on the engarchound ithel bet, any not ming saver eprious Clicht, were basilus to the strees in netion canters of mank hist is a filler, to lengund the pablec. Hard
Halding,” sad the ritk” re
will be mornert wourd nott colmant, andly
apcearing.

Heore, bet. “Anot was wirnaster, af fide.
Hexcrose dea, thing!” reliled the vislaby infear Midntwen


In [33]:
#로짓의 스케일 조정하기
#계수를 점점 작게하면 균등해진다.
logits = np.array([[1.0, 1.0, 3.0]])

print('스케일 조정 전의 확률: ', tf.math.softmax(logits).numpy()[0])

print('0.5배 조정 후 확률:  ', tf.math.softmax(0.5*logits).numpy()[0])

print('0.1배 조정 후 확률:  ', tf.math.softmax(0.1*logits).numpy()[0])

스케일 조정 전의 확률:  [0.10650698 0.10650698 0.78698604]
0.5배 조정 후 확률:   [0.21194156 0.21194156 0.57611688]
0.1배 조정 후 확률:   [0.31042377 0.31042377 0.37915245]


In [34]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=2.0))

The island of the colland the more to the ore had been frear of the corther on the reach of the conting of the sease the onger of the seach had not mad of the vester of the corbers. The will bence to the portan the corsion Harding the callanis plase the corear of the conters of the cortonce in the shouth of the sone day of the age of the said contion’s tate of the ronging to the was and the sive and it of the engineer to the the wase to the colland the sure of have had the contine of the was seas and the 


In [35]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', 
             scale_factor=0.5))

The island eloro? Bib Nof,” rUmmtlawmmoodqhst.

Tha qoadl Thene ourvalnys. Thy? Anklo Cyot IsmecMeces righacking Nil.,” resthoring as the mhansy.

Al?
-Tapnthaweysu.
Helme.
ADo, Frxting tree Tlvalar’! Pepllot obtes! Posstad
ambitk thead
Hhicntatint; sttreis flfh. ashed it!.”
 ole no-Troy eur.
2BGiltersule. PuccCorear ., WapaIn.

CApouctyis
CWoas,,evo” pr vike Nn. Yow?”.
“sivions,”
3n
clithlerkzg
3r cepperw.

The foles,;”
Dusht. Mxtithivetem PeccanfStfy two.

DN Spowarpttcle, hosighetsin.
“If ug na ravoun,


# 16.4 트랜스포머 모델을 사용한 언어 이해

입력과 출력 시퀀스사이에 있는 전역 의존성을 모델링할 수 있다.

어텐션을 기반으로 하고 구체적으로는 셀프 어텐션 메커니즘을 기반으로한다.

## 16.4.1 셀프 어텐션 메커니즘 이해

셀프 어텐션 기본 구조

입력 원소에 대한 출력 시퀀스에 있는 각 원소의 의존성을 모델링하는 것이 목적이다.

세 단계로 구성된다.

첫째, 현재 원소와 시퀀스에 있는 다른 모든 원소 사이의 유사도를 기반으로 중요도 가중치를 계산한다.

둘째, 소프트맥스 함수를 사용하여 이 가중치를 정규화한다.

셋째, 가중치를 해당하는 시퀀스 원소와 결합하여 어텐션 값을 계산한다.

In [36]:
Image(url='https://git.io/JLdVo', width=700)

쿼리, 키, 값 가중치를 가진 셀프 어텐션 메커니즘

셀프 어텐션은 출력을 계산할 때 계산되는 파라미터를 전혀 사용하지 않았다.

따라서 언어 모델을 훈련할 때 분류 오차를 최소화하는 것 같이 목적 함수를 최적화하려면 입력 원소가 되는 단어 임베딩을 바꿔야 한다.

다르게 말해 기본적인 셀프 어텐션 메커니즘을 사용하면 트랜스포머 모델이 주어진 시퀀스에서 모델을 최적화하는 동안 어텐션 값을 바꾸거나 업데이트하는 데 제한적이다.

셀프 어텐션 메커니즘을 모델 최적화에 대해 유연하고 적응할 수 있게 만들기 위해 추가적인 가중치 행렬을 사용하겠다.

## 16.4.2 멀티-헤드 어텐션과 트랜스포머 블록

각 셀프 어텐션 메커니즘을 헤드라고 부르며 병렬로 계산할 수 있다.

r개의 병렬 헤드를 사용하여 각 헤드는 크기가 m인 벡터 h를 만든다. 

이 벡터를 연결하여 크기가 r x m인 z를 얻는다.

마지막으로 이 연결된 벡터와 출력 행렬을 점곱하여 다음과 같이 최종 출력을 만든다.

In [37]:
Image(url='https://git.io/JLdV6', width=700)

아직 언급하지 않은 두 가지 구성 요소가 추가되어 있다.

이 중 하나는 잔차 연결이다.

층의 출력을 입력에 더한다.

잔차 연결로 층을 구성하는 블록을 잔차 블록이라고 한다.

다른 하나는 층 정규화이다.

각 층에서 신경망의 입력과 활성화 출력을 정규화 또는 스케일을 조정하는 고급 방법이다.

동작 방식에 대해 말해보면

먼저 입력 시퀀스가 앞서 언급한 셀프 어텐션 메커니즘을 기반으로 하는 MHA 층으로 전달된다.

또한 입력 시퀀스가 잔차 연결을 통해 MHA 층의 출력에 더해진다.

이렇게 하면 훈련하는 동안 앞쪽의 층이 충분한 그레이디언트 신호를 받게 된다.

훈련 속도와 수렴을 향상시키기 위해 자주 사용되는 기법이다.

입력 시퀀스가 MHA 층의 출력에 더해진 후 이 출력이 층 정규화를 통해 정규화된다.

정규화된 신호가 연속된 MLP 층과 잔차 연결을 통과한다.

마지막으로 잔차 블록의 출력을 다시 정규화하여 출력 시퀀스로 반환한다.
